In [1]:
import os
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Hash import MD5


In [2]:
# WRITING TO FILES
def savePrivateKey(private, size):
    with open("privateKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tPrivate key\n\n")
        f.write("Method:\n")
        f.write("\tRSA\n\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(size)+"\n")
        f.write("\nPrivate key: \n")
        f.write(str(private.exportKey())[2:-1])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def savePublicKey(public, size):
    with open("publicKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tPublic key\n\n")
        f.write("Method:\n")
        f.write("\tRSA\n\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(size)+"\n")
        f.write("\nPublic key: \n")
        f.write(str(public.exportKey())[2:-1])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def saveAESKey(key):
    with open("aesKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tSecret key\n\n")
        f.write("Method:\n")
        f.write("\tAES\n")
        f.write("\nSecret key: \n")
        f.write(str(key))
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")

def printEnvelope(data, encryptedKey, aes_size, rsa_size):
    with open("envelope.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tEnvelope\n\n")
        f.write("Method:\n")
        f.write("\tAES\n")
        f.write("\tRSA\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(aes_size) + "\n")
        f.write("\t0{0:x}".format(rsa_size) + "\n")
        f.write("\nEnvelope data: \n")
        f.write(str(data))
        f.write("\n")
        f.write("\nEnvelope crypt key: \n")
        f.write(str(encryptedKey)[2:-2])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def printSignature(digest, sha_size, rsa_size):
    with open("signature.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tSignature\n\n")
        f.write("Method:\n")
        f.write("\tSHA-1\n")
        f.write("\tRSA\n")
        f.write("\nKey length:\n")
        f.write("\t0{0:x}".format(sha_size) + "\n")
        f.write("\t0{0:x}".format(rsa_size) + "\n")
        f.write("\nSignature: \n")
        f.write(str(digest))
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")

In [3]:
# AES
EncodeAES = lambda c, s: base64.b64encode(c.encrypt(s))
DecodeAES = lambda c, e: c.decrypt(base64.b64decode(e))

# generate a random secret key and initialization vector
secret = os.urandom(16)
IV = os.urandom(16)
print ("Secret key:", str(secret)[2:-1])
saveAESKey(secret)

cipher = AES.new(secret, AES.MODE_CFB, IV)

encoded = EncodeAES(cipher, 'test string')
print ('Encrypted string:', str(encoded)[2:-1])

cipher = AES.new(secret, AES.MODE_CFB, IV)

decoded = DecodeAES(cipher, encoded)
print ('Decrypted string:', str(decoded)[2:-1])

Secret key: {\x12\x0b!\xb45\xa2Y\xc7\xcd\xdf\xfb\x8b\x11\x11\xfe
Encrypted string: lYs0qmnHaEabrxY=
Decrypted string: test string


In [4]:
# RSA

KEY_LENGTH = 1024
random_gen = Random.new().read
 
keypair = RSA.generate(KEY_LENGTH, random_gen)
 
publicKey = keypair.publickey()
savePublicKey(publicKey, 1024)
savePrivateKey(keypair, 1024)
 
print("AES key:", str(secret)[2:-1], "\n")
message = secret

messageHash = MD5.new(message).digest()

signature = keypair.sign(messageHash, '')
 
encryptedMessage = publicKey.encrypt(message, 32)
print ("Encrypted AES key: ", str(encryptedMessage)[3:-3], "\n")
 
decryptedMessage = keypair.decrypt(encryptedMessage)
 
# Signature validation
decryptedHash = MD5.new(decryptedMessage).digest()

if publicKey.verify(decryptedHash, signature):
    print ("Decrypted AES key:", str(decryptedMessage)[2:-1], "\n")

AES key: {\x12\x0b!\xb45\xa2Y\xc7\xcd\xdf\xfb\x8b\x11\x11\xfe 

Encrypted AES key:  *\x821R\x92\xf0$\xd8\xf7\xefS\xa1\xb3\xab\xfbQ\xcd\xca\xa3\xe1TU\xa9\xcd0%\xf5\x14;\xafiekh$Q\x02O\xbc\x1f\xdce\xf0]\xe6\xb0\x1bhW\xbcuP\xb3\xa2WhOp>JO\xaa=\x14\xb9>\xda\x9c\xfd\xce\x96\xcad.\'\x1aM\xa3U\xcf\xa9\xbb\xea\xd2\x963\x97\rA\xc1\x19\xe2P\x9di\xdcA\xcaK!!s^g\x81\xdb\xaa\xcd xA\x8e"\xb5ps\xb0\xb5\x06\xb3o\x9a\xe6\x18\xbaOA\xc5 

Decrypted AES key: {\x12\x0b!\xb45\xa2Y\xc7\xcd\xdf\xfb\x8b\x11\x11\xfe 



In [5]:
# DIGITALNA OMOTNICA

# KRIPTIRANJE
P = "Ovo je moja poruka"
print("Pocetna poruka: ", P, "\n")

AES_BLOCK_SIZE = 16

secret = os.urandom(AES_BLOCK_SIZE)
IV = os.urandom(AES_BLOCK_SIZE)

cipher = AES.new(secret, AES.MODE_CFB, IV)

C1 = EncodeAES(cipher, P)

keypair = RSA.generate(KEY_LENGTH, random_gen)
publicKey = keypair.publickey()

C2 = publicKey.encrypt(secret, 32)

M = (C1, C2)

print("Sadrzaj omotnice M:\n", M, "\n")

printEnvelope(C1, C2, 128, 1024)

# DEKRIPTIRANJE

key = keypair.decrypt(C2)

cipher = AES.new(key, AES.MODE_CFB, IV)

P = DecodeAES(cipher, C1)
print ('Dekriptirana poruka:', P.decode('ascii'))

Pocetna poruka:  Ovo je moja poruka 

Sadrzaj omotnice M:
 (b'5UQsGRJY7XQyB874utH3rt1i', (b"O\x06\x937\xbcD\x8e\xb5\xfb\x81\xf6\x0e\xdb~t\xcbq\xb5\xfc9\xf3\xb1\x83\x83\xbf\xf7\x901\xad8Tm\xdd\xd5\xce\xe4sL0\x86\x07\xd5\xe7\xd5\xc2\x98\x99\xbe\xf3J\xae\x08\xdeF\x8c!\xf9\xc6\x8aBB\xd6+v\xe1GG\xec4\xf2F\xb0\xa4\xbd\xb0\xaa\xe20\xc8\x9a4Ft\xe6\xcf\x90w\xb0\xcff9\x0b.\xfdb\x8a\x9fb\x83\xd2\xc8\x9c\x96j\x04\x99\x98\r!=p\xb0\xf5\x18'\x01\x07&]lN\xa1\x91\xa4\xdf[-~",)) 

Dekriptirana poruka: Ovo je moja poruka


In [6]:
# DIGITALNI POTPIS

m = hashlib.md5()

m.update(str(C1).encode('utf-8'))
m.update(str(C2).encode('utf-8'))

H = m.digest()
hexdigest = m.hexdigest()

printSignature(hexdigest, 160, 1024)

print("Digest:", str(H)[2:-1])
print("Hex digest:", hexdigest)

Digest: Qg%\xb2\x9e\xa9IK\xe9\xfd{\xdcI\xd5}\x12
Hex digest: 516725b29ea9494be9fd7bdc49d57d12


In [7]:
# DIGITALNI PECAT

keypair = RSA.generate(KEY_LENGTH, random_gen)
publicKey = keypair.publickey()
secret = os.urandom(AES_BLOCK_SIZE)
IV = os.urandom(16)

cipher = AES.new(secret, AES.MODE_CFB, IV)

C1 = EncodeAES(cipher, P)
C3 =  publicKey.encrypt(H, 32)
C2 = publicKey.encrypt(secret, 32)

M = (C1, C2, C3)

print("M(C1, C2, C3):", M, "\n")

K = keypair.decrypt(C2)
cipher = AES.new(K, AES.MODE_CFB, IV)
P = DecodeAES(cipher, C1)
S = keypair.decrypt(C3)

print ("K=", str(K)[2:-1])
print ("P=", str(P)[2:-1])
print ("S=", str(S)[2:-1])

M(C1, C2, C3): (b'z5PqSMwzahnvZR9vWjaf5fVg', (b'\xbf{\xabX6\xe0P\xfc)\xbee\xd3\x82I\xcc\x0e\xefl\xaf\xbfz\xc1\x9c\x108\xd20\x07k\x14A\xc7c\x07>\x0f)\xe4\x06\xd4y\xb93P\xf9y0\xa7p\xeb\xcc*\x16\xf87Ht\xa9\xca\xb1^!\x1cf4\x0b\xe5\x17\xcb\xf9\xdc\x1e\x05\xd7\xfe\x06[\x8f\xd7\xdf\xbe\'\xf6a\x9e~\xef\x08\xacj\xef\x95uVE\xd8/E"7\xa3e\xb7\x98\x9d\x9fET\t\xefNW\x96P\x01X;\t>\xf2\x97:a;3e\x03\x1e',), (b'\x0f\xf2\x14\xf5\x13\xc0\xdcT\xee6d\xbd3\xf3\x15\x89\xf4\x05\x14\xf3~\xb0o4\x8c\xae\x8f\x8b3\xd6}=@0|\xa7U\xe6\x93\xb8\xf9\x83]\x19\xc1\xa8\x07*C\xd6JPt\xb7>\xed\x96\xde\xb1\xfd\xe8|N\xd8\xfe\x19\x9d\xc6X\xc5\xbf\xe6s\xa6\xf2\xfdji\xb8BE\xe8}\xe7\xcc\xcb0\xa5=\xe7g\rb\x94f\x88C\x89$f\xd8ug\xb31BN\x08\x08y\xef\xc4\x05\xb26\x1b\xde@\xe6\x0fv\xd2\x92\xf7.\x81\xb4+',)) 

K= 2l\xc7\xf4D\xd4k\x1d\xcaZ\x1fgp\x1d\x96R
P= Ovo je moja poruka
S= Qg%\xb2\x9e\xa9IK\xe9\xfd{\xdcI\xd5}\x12


In [8]:
# SHA-1 IMPLEMENTATION

"""
Razbija poruke u komade od n bitova.
"""
def chunks(array, n):
        return [array[i:i+n] for i in range(0, len(array), n)]

"""
Rotacija dobivenog niza u lijevo za n bitova.
"""
def rotate_left(word, n):
    return ((word << n) | (word >> (32 - n))) & 0xffffffff
    
def sha1(data):
    bytes = ""
    
    # INICIJALIZACIJA VARIJABLE
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0
    
    # PRED-OBRADA
    
    # pretvaranje ulaznog stringa u 8-bitni zapis
    for n in range(len(data)):
        bytes += '{0:08b}'.format(ord(data[n]))
    
    # dodavanje 1 na kraj poruke
    bits = bytes + "1"

    paddedBits = bits
    
    # dodavanje k 0 sve dok rezultirajuca poruke bude kongruent 448 (mod 512)
    while len(paddedBits) % 512 != 448:
        paddedBits += "0"
        
    # dodaj duljinu pocetnog niza kao 64-bitni big-endidan cijeli broj
    paddedBits += '{0:064b}'.format(len(bits) - 1)
    
    # PODJELA NA DIJELOVE I OBRADA
    for c in chunks(paddedBits, 512): 
        words = chunks(c, 32)
        w = [0]*80

        for n in range(0, 16):
            w[n] = int(words[n], 2)
            
        for i in range(16, 80):
            w[i] = rotate_left((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]), 1)  

        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        for i in range(0, 80):
            if 0 <= i <= 19:
                f = (b & c) | ((~b) & d)
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d) 
                k = 0x8F1BBCDC
            elif 60 <= i <= 79:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = rotate_left(a, 5) + f + e + k + w[i] & 0xffffffff
            
            e = d
            d = c
            c = rotate_left(b, 30)
            b = a
            a = temp

        h0 = h0 + a & 0xffffffff
        h1 = h1 + b & 0xffffffff
        h2 = h2 + c & 0xffffffff
        h3 = h3 + d & 0xffffffff
        h4 = h4 + e & 0xffffffff

    return '%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4)

In [9]:
m = hashlib.sha1()
m.update("ovo je test string".encode('utf-8'))
print(m.hexdigest())
print(sha1("ovo je test string"))

7ed1c126db54c5dc6fee95f6db1e699177fa05d2
7ed1c126db54c5dc6fee95f6db1e699177fa05d2
